# 第二章 路由查询引擎 Router Query Engine

## 一、引言

在本章节中，我们将学习最简单形式的<strong>代理式检索增强生成(Agentic RAG)</strong>。给定一个<strong>查询(Query)</strong>，<strong>路由器(Router)</strong>通过理解查询，从几个<strong>查询引擎(Query Engine)</strong>中的选择一个来执行查询。

我们将构建一个简单的路由器，该路由器可以处理<strong>单个文档的问答和摘要</strong>。 

In [2]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio

nest_asyncio.apply()

## 二、加载数据

这里需要用到的文档为`metagpt.pdf`。已经在当前目录下面。 或者你也可以通过下面的指令来下载文件。

```bash
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf
```

`wget` 为linux系统下的下载指令。如果你的系统为Windows或者Mac, 则可以通过curl来下载。

```bash
!curl -s -o metagpt.pdf "https://openreview.net/pdf?id=VtmBAGCN7o"
```


### 2.1 加载数据为文档格式

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [4]:
type(documents)

list

In [5]:
len(documents)

29

<br>

数据加载后的为格式为列表(`List`)。这里包含列表中共有29个元素。

我门来看看元素的类型。
<br>

In [6]:
type(documents[0])

llama_index.core.schema.Document

<br>

元素的类型为llama_index自定义文档类型(`llama_index.core.schema.Document`)

<br>

In [7]:
documents[0].to_dict().keys()

dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])

<br>

将llama_index自定义文档类型转为为字典类型（`dictionary`），并输出其中的键(`keys`)。可以看到对于每个文档，包含的信息。

<br>

In [8]:
print("------------------------------------")
print("文档的字符个数")
print("------------------------------------")
print([len(documents[i].text) for i in range(len(documents))])
print()

------------------------------------
文档的字符个数
------------------------------------
[3612, 2866, 4602, 2757, 1583, 4280, 3656, 1925, 3265, 3884, 3605, 3505, 3357, 1160, 4326, 818, 2475, 1285, 1837, 901, 928, 1943, 2274, 3014, 3307, 1522, 255, 2668, 1726]



### 2.2 切割文档为节点格式

In [9]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [10]:
len(nodes)

34

In [11]:
type(nodes[0])

llama_index.core.schema.TextNode

In [12]:
nodes[0].to_dict().keys()

dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])

In [13]:
print("------------------------------------")
print("节点的字符个数")
print("------------------------------------")
print([len(nodes[i].text) for i in range(len(nodes))])
print()
print("----------------------------------------------------------------")
print("第三个文档被分隔为两个节点,紫色+绿色为第一个节点，绿色+蓝色为第二个节点。")
print("也就是说，绿色的部分同时在两个节点中")
print("----------------------------------------------------------------")
from src.utils import highlight_doc

highlight_doc(doc = documents[2].text, 
              splited_node1=nodes[2].text, 
              splited_node2=nodes[3].text)


------------------------------------
节点的字符个数
------------------------------------
[3612, 2866, 4102, 1309, 2757, 1583, 4280, 3450, 328, 1925, 3265, 2924, 1532, 3075, 1172, 3154, 928, 3357, 1160, 4326, 818, 2475, 1285, 1837, 901, 928, 1943, 2274, 3014, 3307, 1522, 255, 2668, 1726]

----------------------------------------------------------------
第三个文档被分隔为两个节点,紫色+绿色为第一个节点，绿色+蓝色为第二个节点。
也就是说，绿色的部分同时在两个节点中
----------------------------------------------------------------
Preprint
•We introduce MetaGPT, a meta-programming framework for multi-agent collaboration based on
LLMs. It is highly convenient and flexible, with well-defined functions like role definition and
message sharing, making it a useful platform for developing LLM-based multi-agent systems.
•Our innovative integration of human-like SOPs throughout MetaGPT’s design significantly en-
hances its robustness, reducing unproductive collaboration among LLM-based agents. Furthermore,
we introduce a novel executive feedback mechanism tha

## 三、定义模型

In [14]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## 四、创建摘要索引和向量索引

基于前面加载并切割得到的节点(nodes)来创建摘要索引和向量索引

In [15]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## 五、定义查询引擎

和设置元数据

In [16]:
from llama_index.core.tools import QueryEngineTool

# 基于摘要索引summary_index构建摘要查询引擎
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# 基于摘要索引summary_index构建向量查询引擎
vector_query_engine = vector_index.as_query_engine()


# 构建摘要查询引擎工具
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

# 构建摘向量查询引擎工具
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## 五、定义路由查询引擎

路由查询引擎从多个候选查询引擎中选择一个来执行查询。
- 选择器(selector): 根据每个候选查询的元数据和查询选择一个选项的选择器。
- 候选查询引擎工具（query_engine_tools）: 一系列候选查询引擎。

In [17]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [18]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework for multi-agent collaboration based on Large Language Models (LLMs). It incorporates Standardized Operating Procedures (SOPs) to enhance collaboration efficiency and reduce errors among agents. MetaGPT assigns specific roles to agents, streamlining workflows and improving code generation quality. Through structured communication interfaces and executable feedback mechanisms, MetaGPT achieves state-of-the-art performance in evaluations, demonstrating its robustness and efficiency in developing LLM-based multi-agent systems. Additionally, the document discusses the software development process using MetaGPT, outlining the roles of different agents in generating requirements, designing systems, developing code, and testing applications. It also covers the performance evaluation of GPT models on the HumanEval benchmark, emphasizing the benefits of M

In [19]:
print(len(response.source_nodes))

34


In [20]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context from the MetaGPT paper, which would likely include information on how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool. This shared message pool allows all agents to exchange messages directly. Agents publish their structured messages in the pool and can also access messages from other entities transparently. This system enables any agent to retrieve required information directly from the shared pool, eliminating the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.


## 六、结合起来

In [3]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [4]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results provide insights into the impact of removing certain components or features from a system or model. This analysis helps in understanding the contribution and importance of individual elements towards the overall performance or functionality of the system.
